In [1]:
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.models as models
# from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 200
%config InlineBackend.figure_format = 'retina'
train_on_gpu = torch.cuda.is_available()

In [2]:

class SeparableConv2d(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size=1,stride=1,padding=0,dilation=1,bias=False):
        super(SeparableConv2d,self).__init__()

        self.conv1 = nn.Conv2d(in_channels,in_channels,kernel_size,stride,padding,dilation,groups=in_channels,bias=bias)
        self.pointwise = nn.Conv2d(in_channels,out_channels,1,1,0,1,1,bias=bias)

    def forward(self,x):
        x = self.conv1(x)
        x = self.pointwise(x)
        return x


class Block(nn.Module):
    def __init__(self,in_filters,out_filters,reps,strides=1,start_with_relu=True,grow_first=True):
        super(Block, self).__init__()

        if out_filters != in_filters or strides!=1:
            self.skip = nn.Conv2d(in_filters,out_filters,1,stride=strides, bias=False)
            self.skipbn = nn.BatchNorm2d(out_filters)
        else:
            self.skip=None

        rep=[]

        filters=in_filters
        if grow_first:
            rep.append(nn.ReLU(inplace=True))
            rep.append(SeparableConv2d(in_filters,out_filters,3,stride=1,padding=1,bias=False))
            rep.append(nn.BatchNorm2d(out_filters))
            filters = out_filters

        for i in range(reps-1):
            rep.append(nn.ReLU(inplace=True))
            rep.append(SeparableConv2d(filters,filters,3,stride=1,padding=1,bias=False))
            rep.append(nn.BatchNorm2d(filters))

        if not grow_first:
            rep.append(nn.ReLU(inplace=True))
            rep.append(SeparableConv2d(in_filters,out_filters,3,stride=1,padding=1,bias=False))
            rep.append(nn.BatchNorm2d(out_filters))

        if not start_with_relu:
            rep = rep[1:]
        else:
            rep[0] = nn.ReLU(inplace=False)

        if strides != 1:
            rep.append(nn.MaxPool2d(3,strides,1))
        self.rep = nn.Sequential(*rep)

    def forward(self,inp):
        x = self.rep(inp)

        if self.skip is not None:
            skip = self.skip(inp)
            skip = self.skipbn(skip)
        else:
            skip = inp

        x+=skip
        return x


class Xception(nn.Module):
    """
    Xception optimized for the ImageNet dataset, as specified in
    https://arxiv.org/pdf/1610.02357.pdf
    """
    def __init__(self, num_classes=1000):
        """ Constructor
        Args:
            num_classes: number of classes
        """
        super(Xception, self).__init__()
        self.num_classes = num_classes

        self.conv1 = nn.Conv2d(3, 32, 3,2, 0, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(32,64,3,bias=False)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU(inplace=True)
        #do relu here

        self.block1=Block(64,128,2,2,start_with_relu=False,grow_first=True)
        self.block2=Block(128,256,2,2,start_with_relu=True,grow_first=True)
        self.block3=Block(256,728,2,2,start_with_relu=True,grow_first=True)

        self.block4=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block5=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block6=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block7=Block(728,728,3,1,start_with_relu=True,grow_first=True)

        self.block8=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block9=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block10=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block11=Block(728,728,3,1,start_with_relu=True,grow_first=True)

        self.block12=Block(728,1024,2,2,start_with_relu=True,grow_first=False)

        self.conv3 = SeparableConv2d(1024,1536,3,1,1)
        self.bn3 = nn.BatchNorm2d(1536)
        self.relu3 = nn.ReLU(inplace=True)

        #do relu here
        self.conv4 = SeparableConv2d(1536,2048,3,1,1)
        self.bn4 = nn.BatchNorm2d(2048)

        self.fc = nn.Linear(2048, num_classes)

        # #------- init weights --------
        # for m in self.modules():
        #     if isinstance(m, nn.Conv2d):
        #         n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        #         m.weight.data.normal_(0, math.sqrt(2. / n))
        #     elif isinstance(m, nn.BatchNorm2d):
        #         m.weight.data.fill_(1)
        #         m.bias.data.zero_()
        # #-----------------------------

    def features(self, input):
        x = self.conv1(input)
        x = self.bn1(x)
        x = self.relu1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)

        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.block6(x)
        x = self.block7(x)
        x = self.block8(x)
        x = self.block9(x)
        x = self.block10(x)
        x = self.block11(x)
        x = self.block12(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)

        x = self.conv4(x)
        x = self.bn4(x)
        return x

    def logits(self, features):
        x = nn.ReLU(inplace=True)(features)

        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = x.view(x.size(0), -1)
        x = self.last_linear(x)
        return x

    def forward(self, input):
        x = self.features(input)
        x = self.logits(x)
        return x


def xception(num_classes=2):
    model = Xception(num_classes=num_classes)
    model.last_linear = model.fc
    del model.fc
    return model.cuda()

In [3]:
class DenseNetBottleneck(nn.Module):
    def __init__(self, nChannels, growthRate):
        super(DenseNetBottleneck, self).__init__()
        interChannels = 4*growthRate
        self.bn1 = nn.BatchNorm2d(nChannels)
        self.conv1 = nn.Conv2d(nChannels, interChannels, kernel_size=1,
                               bias=False)
        self.bn2 = nn.BatchNorm2d(interChannels)
        self.conv2 = nn.Conv2d(interChannels, growthRate, kernel_size=3,
                               padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = self.conv2(F.relu(self.bn2(out)))
        out = torch.cat((x, out), 1)
        return out

class SingleLayer(nn.Module):
    def __init__(self, nChannels, growthRate):
        super(SingleLayer, self).__init__()
        self.bn1 = nn.BatchNorm2d(nChannels)
        self.conv1 = nn.Conv2d(nChannels, growthRate, kernel_size=3,
                               padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = torch.cat((x, out), 1)
        return out

class Transition(nn.Module):
    def __init__(self, nChannels, nOutChannels):
        super(Transition, self).__init__()
        self.bn1 = nn.BatchNorm2d(nChannels)
        self.conv1 = nn.Conv2d(nChannels, nOutChannels, kernel_size=1,
                               bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = F.avg_pool2d(out, 2)
        return out


class DenseNet(nn.Module):
    def __init__(self, growthRate, depth, reduction, nClasses, bottleneck):
        super(DenseNet, self).__init__()

        nDenseBlocks = (depth-4) // 3
        if bottleneck:
            nDenseBlocks //= 2

        nChannels = 2*growthRate
        self.conv1 = nn.Conv2d(3, nChannels, kernel_size=3, padding=1,
                               bias=False)
        self.dense1 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        nChannels += nDenseBlocks*growthRate
        nOutChannels = int(math.floor(nChannels*reduction))
        self.trans1 = Transition(nChannels, nOutChannels)

        nChannels = nOutChannels
        self.dense2 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        nChannels += nDenseBlocks*growthRate
        nOutChannels = int(math.floor(nChannels*reduction))
        self.trans2 = Transition(nChannels, nOutChannels)

        nChannels = nOutChannels
        self.dense3 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        nChannels += nDenseBlocks*growthRate

        self.bn1 = nn.BatchNorm2d(nChannels)
        self.fc = nn.Linear(nChannels, nClasses)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()

    def _make_dense(self, nChannels, growthRate, nDenseBlocks, bottleneck):
        layers = []
        for i in range(int(nDenseBlocks)):
            if bottleneck:
                layers.append(DenseNetBottleneck(nChannels, growthRate))
            else:
                layers.append(SingleLayer(nChannels, growthRate))
            nChannels += growthRate
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.trans1(self.dense1(out))
        out = self.trans2(self.dense2(out))
        out = self.dense3(out)
        out = torch.squeeze(F.avg_pool2d(F.relu(self.bn1(out)), 8))
        out = F.log_softmax(self.fc(out))
        return out



In [4]:
def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=True)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNetBottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(ResNetBottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=True)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=True)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=True)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=2):
        self.inplanes = 64
        # Special attributs
        self.input_space = None
        self.input_size = (299, 299, 3)
        self.mean = None
        self.std = None
        super(ResNet, self).__init__()
        # Modules
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                                bias=True)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7)
        self.last_linear = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=True),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def features(self, input):
        x = self.conv1(input)
        self.conv1_input = x.clone()
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x

    def logits(self, features):
        x = self.avgpool(features)
        x = x.view(x.size(0), -1)
        x = self.last_linear(x)
        return x

    def forward(self, input):
        x = self.features(input)
        x = self.logits(x)
        return x


def ResNet34():
    model = ResNet(BasicBlock, [3, 4, 6, 3], num_classes=2)
    # model.load_state_dict(torch.load(path_to_pretrained))
    return model.cuda()

In [5]:
Xception_car = xception(num_classes=2)
Xception_car = nn.DataParallel(Xception_car)
Xception_car.load_state_dict(torch.load('Xception_car_best.pt'))

Xception_cat = xception(num_classes=2)
Xception_cat = nn.DataParallel(Xception_cat)
Xception_cat.load_state_dict(torch.load('Xception_cat_best.pt'))

Xception_horse = xception(num_classes=2)
Xception_horse = nn.DataParallel(Xception_horse)
Xception_horse.load_state_dict(torch.load('Xception_horse_best.pt'))

Xception_person = xception(num_classes=2)
Xception_person = nn.DataParallel(Xception_person)
Xception_person.load_state_dict(torch.load('Xception_person_best.pt'))

Xception_sofa = xception(num_classes=2)
Xception_sofa = nn.DataParallel(Xception_sofa)
Xception_sofa.load_state_dict(torch.load('Xception_sofa_best.pt'))

DenseNet_car = DenseNet(growthRate=24, depth=100, reduction=0.5,bottleneck=True, nClasses=2).cuda()
DenseNet_car = nn.DataParallel(DenseNet_car)
DenseNet_car.load_state_dict(torch.load('DenseNet_car_best.pt'))

DenseNet_cat = DenseNet(growthRate=24, depth=100, reduction=0.5,bottleneck=True, nClasses=2).cuda()
DenseNet_cat = nn.DataParallel(DenseNet_cat)
DenseNet_cat.load_state_dict(torch.load('DenseNet_cat_best.pt'))

DenseNet_horse = DenseNet(growthRate=24, depth=100, reduction=0.5,bottleneck=True, nClasses=2).cuda()
DenseNet_horse = nn.DataParallel(DenseNet_horse)
DenseNet_horse.load_state_dict(torch.load('DenseNet_horse_best.pt'))

DenseNet_person = DenseNet(growthRate=24, depth=100, reduction=0.5,bottleneck=True, nClasses=2).cuda()
DenseNet_person = nn.DataParallel(DenseNet_person)
DenseNet_person.load_state_dict(torch.load('DenseNet_person_best.pt'))

DenseNet_sofa = DenseNet(growthRate=24, depth=100, reduction=0.5,bottleneck=True, nClasses=2).cuda()
DenseNet_sofa = nn.DataParallel(DenseNet_sofa)
DenseNet_sofa.load_state_dict(torch.load('DenseNet_sofa_best.pt'))

Spec_car = ResNet34()
Spec_car = nn.DataParallel(Spec_car)
Spec_car.load_state_dict(torch.load('Spec_car_best.pt'))

Spec_cat = ResNet34()
Spec_cat = nn.DataParallel(Spec_cat)
Spec_cat.load_state_dict(torch.load('Spec_cat_best.pt'))

Spec_horse = ResNet34()
Spec_horse = nn.DataParallel(Spec_horse)
Spec_horse.load_state_dict(torch.load('Spec_horse_best.pt'))

Spec_person = ResNet34()
Spec_person = nn.DataParallel(Spec_person)
Spec_person.load_state_dict(torch.load('Spec_person_best.pt'))

Spec_sofa = ResNet34()
Spec_sofa = nn.DataParallel(Spec_sofa)
Spec_sofa.load_state_dict(torch.load('Spec_sofa_best.pt'))

ResNet34DCT_car = ResNet34()
ResNet34DCT_car = nn.DataParallel(ResNet34DCT_car)
ResNet34DCT_car.load_state_dict(torch.load('ResNet34DCT_car_best.pt'))

ResNet34DCT_cat = ResNet34()
ResNet34DCT_cat = nn.DataParallel(ResNet34DCT_cat)
ResNet34DCT_cat.load_state_dict(torch.load('ResNet34DCT_cat_best.pt'))

ResNet34DCT_horse = ResNet34()
ResNet34DCT_horse = nn.DataParallel(ResNet34DCT_horse)
ResNet34DCT_horse.load_state_dict(torch.load('ResNet34DCT_horse_best.pt'))

ResNet34DCT_person = ResNet34()
ResNet34DCT_person = nn.DataParallel(ResNet34DCT_person)
ResNet34DCT_person.load_state_dict(torch.load('ResNet34DCT_person_best.pt'))

ResNet34DCT_sofa = ResNet34()
ResNet34DCT_sofa = nn.DataParallel(ResNet34DCT_sofa)
ResNet34DCT_sofa.load_state_dict(torch.load('ResNet34DCT_sofa_best.pt'))

<All keys matched successfully>

In [6]:
def XceptionTestConstructor(path_to_test, classes, num_workers=4):
    
  
    # Transformations to the image, edit as need be
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.Resize([299,299]),
        transforms.ToTensor()])

    test_dataset = datasets.ImageFolder(path_to_test, transform=transform)
#     print("Successfully Loaded Test Set.")
    
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=2, 
        num_workers=num_workers)
#     if classes != None:
#         print("Number of Classes:", len(classes))
    return test_loader, classes

In [7]:
def DenseNetTestConstructor(path_to_test, classes=None, num_workers=4):

    test_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.Resize([32,32]),
        transforms.ToTensor()])

    test_dataset = datasets.ImageFolder(path_to_test, transform=test_transform)
#     print("Successfully Loaded Test Set.")

    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=2, num_workers=num_workers)

#     if classes != None:
#         print("Number of Classes:", len(classes))

    return test_loader, classes

In [8]:
def SpectrumTestConstructor(path_to_test, classes=None, num_workers=4):

    test_transform = transforms.Compose([
        transforms.Resize([224, 224]),
        transforms.ToTensor()
        ])

    test_dataset = datasets.ImageFolder(path_to_test, transform=test_transform)
#     print("Successfully Loaded Test Set.")

    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=2, num_workers=num_workers)

#     if classes != None:
#         print("Number of Classes:", len(classes))

    return test_loader, classes

In [9]:
convert = nn.Softmax(1)

In [ ]:
test_categories = ['airplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow','diningtable','dog','horse','motorbike','person','pottedplant','sheep','sofa','train','tvmonitor']

for k in range(20):
    print('For test category ' + test_categories[k] + ':')

    xcp_test_loader, image_classes = XceptionTestConstructor(path_to_test='F:/data/testset/progan/'+ test_categories[k],
                                                               classes=['Real','Fake'],num_workers=1)
    dsn_test_loader, image_classes = DenseNetTestConstructor(path_to_test='F:/data/testset/progan/'+ test_categories[k],
                                                               classes=['Real','Fake'],num_workers=1)
    fft_test_loader, image_classes = SpectrumTestConstructor(path_to_test='C:/Users/22710/Desktop/DCT/FFT/CNN_synth_testset/progan/'
                                                             + test_categories[k],classes=['Real','Fake'],num_workers=1)
    dct_test_loader, image_classes = SpectrumTestConstructor(path_to_test='C:/Users/22710/Desktop/DCT/CNN_synth_testset/progan/'
                                                             + test_categories[k],classes=['Real','Fake'],num_workers=1)

    car_model = []
    test_loader_all = []
    test_loader_all.append(xcp_test_loader)
    test_loader_all.append(dsn_test_loader)
    test_loader_all.append(fft_test_loader)
    test_loader_all.append(dct_test_loader)
    car_model.append(Xception_sofa)
    car_model.append(DenseNet_sofa)
    car_model.append(Spec_sofa)
    car_model.append(ResNet34DCT_sofa)

    weights_real = []
    weights_fake = []
    target_all = []
    for j in range(4):
        weight_real = []
        weight_fake = []
        target_single = []
        model = car_model[j]
        model.eval()
        # specify loss function (categorical cross-entropy)
        criterion = nn.CrossEntropyLoss()

        # specify optimizer
        optimizer = torch.optim.SGD([
            {'params': list(model.parameters())[:-1], 'lr': 1e-3, 'momentum': 0.9, 'weight_decay': 1e-3},
            {'params': list(model.parameters())[-1], 'lr': 5e-5, 'momentum': 0.9, 'weight_decay': 1e-5}
        ])
        # iterate over test data
        for data, target in test_loader_all[j]:
        #     print(target)
            # move tensors to GPU if CUDA is available
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
        #     print(output)
            # calculate the batch loss
            loss = criterion(output, target)

            # compare predictions to true label
            probabilities = convert(output)
            num_output = probabilities.cpu().detach().numpy()
            num_target = target.cpu().detach().numpy()
            for i in range(2):
                weight_real.append(num_output[i][0])
                weight_fake.append(num_output[i][1])
                target_single.append(num_target[i])

        weights_real.append(weight_real)
        weights_fake.append(weight_fake)
        target_all.append(target_single)
    
    count_real_correct = 0
    count_fake_correct = 0
    for i in range(400):
        weighted_real = (weights_real[0][i] + weights_real[1][i] + weights_real[2][i] + weights_real[3][i])/4
        weighted_fake = (weights_fake[0][i] + weights_fake[1][i] + weights_fake[2][i] + weights_fake[3][i])/4
        if weighted_real > weighted_fake:
            if i < 200:
                count_real_correct += 1
        else:
            if i >= 200:
                count_fake_correct += 1
    
    print('Test Accuracy of Real: %2d%% (%2d/%2d)'%(100*count_real_correct/200, count_real_correct, 200))
    print('Test Accuracy of Fake: %2d%% (%2d/%2d)'%(100*count_fake_correct/200, count_fake_correct, 200))
    print('Test Accuracy (Overall): %2d%% (%2d/%2d)'%(100*(count_real_correct+count_fake_correct)/400, 
                                                      count_real_correct+count_fake_correct, 400))
    #     # average test loss)
    #     test_loss = test_loss/len(test_loader.dataset)
    #     print('Test Loss: {:.6f}'.format(test_loss))

    #     for i in range(2):
    #         if class_total[i] > 0:
    #             print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
    #                 classes[i], 100 * class_correct[i] / class_total[i],
    #                 np.sum(class_correct[i]), np.sum(class_total[i])))
    #         else:
    #             print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

    #     print('Test Accuracy (Overall): %2d%% (%2d/%2d)\n' % (
    #         100. * np.sum(class_correct) / np.sum(class_total),
    #         np.sum(class_correct), np.sum(class_total)))

In [21]:
xcp_test_loader, image_classes = XceptionTestConstructor(path_to_test='F:/data/testset/deepfake',
                                                           classes=['Real','Fake'],num_workers=1)
dsn_test_loader, image_classes = DenseNetTestConstructor(path_to_test='F:/data/testset/deepfake',
                                                           classes=['Real','Fake'],num_workers=1)
fft_test_loader, image_classes = SpectrumTestConstructor(path_to_test='C:/Users/22710/Desktop/DCT/FFT/CNN_synth_testset/deepfake'
                                                         ,classes=['Real','Fake'],num_workers=1)
dct_test_loader, image_classes = SpectrumTestConstructor(path_to_test='C:/Users/22710/Desktop/DCT/CNN_synth_testset/deepfake'
                                                         ,classes=['Real','Fake'],num_workers=1)

car_model = []
test_loader_all = []
test_loader_all.append(xcp_test_loader)
test_loader_all.append(dsn_test_loader)
test_loader_all.append(fft_test_loader)
test_loader_all.append(dct_test_loader)
car_model.append(Xception_sofa)
car_model.append(DenseNet_sofa)
car_model.append(Spec_sofa)
car_model.append(ResNet34DCT_sofa)

weights_real = []
weights_fake = []
target_all = []
for j in range(4):
    weight_real = []
    weight_fake = []
    target_single = []
    model = car_model[j]
    model.eval()
    # specify loss function (categorical cross-entropy)
    criterion = nn.CrossEntropyLoss()

    # specify optimizer
    optimizer = torch.optim.SGD([
        {'params': list(model.parameters())[:-1], 'lr': 1e-3, 'momentum': 0.9, 'weight_decay': 1e-3},
        {'params': list(model.parameters())[-1], 'lr': 5e-5, 'momentum': 0.9, 'weight_decay': 1e-5}
    ])
    # iterate over test data
    for data, target in test_loader_all[j]:
    #     print(target)
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
    #     print(output)
        # calculate the batch loss
        loss = criterion(output, target)

        # compare predictions to true label
        probabilities = convert(output)
        num_output = probabilities.cpu().detach().numpy()
        num_target = target.cpu().detach().numpy()
        for i in range(2):
            weight_real.append(num_output[i][0])
            weight_fake.append(num_output[i][1])
            target_single.append(num_target[i])

    weights_real.append(weight_real)
    weights_fake.append(weight_fake)
    target_all.append(target_single)

count_real_correct = 0
count_fake_correct = 0
num_real = target_all[0].count(0)
num_fake = target_all[0].count(1)
for i in range(num_real+num_fake):
    weighted_real = (weights_real[0][i] + weights_real[1][i] + weights_real[2][i] + weights_real[3][i])/4
    weighted_fake = (weights_fake[0][i] + weights_fake[1][i] + weights_fake[2][i] + weights_fake[3][i])/4
    if weighted_real > weighted_fake:
        if i < num_real:
            count_real_correct += 1
    else:
        if i >= num_real:
            count_fake_correct += 1

print('Test Accuracy of Real: %2d%% (%2d/%2d)'%(100*count_real_correct/num_real, count_real_correct, num_real))
print('Test Accuracy of Fake: %2d%% (%2d/%2d)'%(100*count_fake_correct/num_fake, count_fake_correct, num_fake))
print('Test Accuracy (Overall): %2d%% (%2d/%2d)'%(100*(count_real_correct+count_fake_correct)/(num_real+num_fake), 
                                                  count_real_correct+count_fake_correct, num_real+num_fake))

C:\Users\22710\AppData\Local\Temp/ipykernel_7840/1079251366.py:98: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = F.log_softmax(self.fc(out))


Test Accuracy of Real: 81% (2200/2706)
Test Accuracy of Fake:  5% (156/2698)
Test Accuracy (Overall): 43% (2356/5404)


In [14]:
test_categories = ['apple','horse','orange','summer','winter','zebra']
# test_categories = ['bedroom','car','cat']
# test_categories = ['car','cat','church','horse']
# test_categories = ['apple']
for k in range(6):
    print('For test category ' + test_categories[k] + ':')

    xcp_test_loader, image_classes = XceptionTestConstructor(path_to_test='F:/data/testset/cyclegan/'+ test_categories[k],
                                                               classes=['Real','Fake'],num_workers=1)
    dsn_test_loader, image_classes = DenseNetTestConstructor(path_to_test='F:/data/testset/cyclegan/'+ test_categories[k],
                                                               classes=['Real','Fake'],num_workers=1)
    fft_test_loader, image_classes = SpectrumTestConstructor(path_to_test='C:/Users/22710/Desktop/DCT/FFT/CNN_synth_testset/cyclegan/'
                                                             + test_categories[k],classes=['Real','Fake'],num_workers=1)
    dct_test_loader, image_classes = SpectrumTestConstructor(path_to_test='C:/Users/22710/Desktop/DCT/CNN_synth_testset/cyclegan/'
                                                             + test_categories[k],classes=['Real','Fake'],num_workers=1)

    car_model = []
    test_loader_all = []
    test_loader_all.append(xcp_test_loader)
    test_loader_all.append(dsn_test_loader)
    test_loader_all.append(fft_test_loader)
    test_loader_all.append(dct_test_loader)
    car_model.append(Xception_sofa)
    car_model.append(DenseNet_sofa)
    car_model.append(Spec_sofa)
    car_model.append(ResNet34DCT_sofa)

    weights_real = []
    weights_fake = []
    target_all = []
    for j in range(4):
        weight_real = []
        weight_fake = []
        target_single = []
        model = car_model[j]
        model.eval()
        # specify loss function (categorical cross-entropy)
        criterion = nn.CrossEntropyLoss()

        # specify optimizer
        optimizer = torch.optim.SGD([
            {'params': list(model.parameters())[:-1], 'lr': 1e-3, 'momentum': 0.9, 'weight_decay': 1e-3},
            {'params': list(model.parameters())[-1], 'lr': 5e-5, 'momentum': 0.9, 'weight_decay': 1e-5}
        ])
        # iterate over test data
        for data, target in test_loader_all[j]:
        #     print(target)
            # move tensors to GPU if CUDA is available
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
        #     print(output)
            # calculate the batch loss
            loss = criterion(output, target)

            # compare predictions to true label
            probabilities = convert(output)
            num_output = probabilities.cpu().detach().numpy()
            num_target = target.cpu().detach().numpy()
            for i in range(2):
                weight_real.append(num_output[i][0])
                weight_fake.append(num_output[i][1])
                target_single.append(num_target[i])

        weights_real.append(weight_real)
        weights_fake.append(weight_fake)
        target_all.append(target_single)
    
    count_real_correct = 0
    count_fake_correct = 0
    num_real = target_all[0].count(0)
    num_fake = target_all[0].count(1)
    for i in range(num_real+num_fake):
        weighted_real = (weights_real[0][i] + weights_real[1][i] + weights_real[2][i] + weights_real[3][i])/4
        weighted_fake = (weights_fake[0][i] + weights_fake[1][i] + weights_fake[2][i] + weights_fake[3][i])/4
        if weighted_real > weighted_fake:
            if i < num_real:
                count_real_correct += 1
        else:
            if i >= num_real:
                count_fake_correct += 1
    
    print('Test Accuracy of Real: %2d%% (%2d/%2d)'%(100*count_real_correct/num_real, count_real_correct, num_real))
    print('Test Accuracy of Fake: %2d%% (%2d/%2d)'%(100*count_fake_correct/num_fake, count_fake_correct, num_fake))
    print('Test Accuracy (Overall): %2d%% (%2d/%2d)'%(100*(count_real_correct+count_fake_correct)/(num_real+num_fake), 
                                                      count_real_correct+count_fake_correct, num_real+num_fake))

For test category apple:


C:\Users\22710\AppData\Local\Temp/ipykernel_8832/1079251366.py:98: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = F.log_softmax(self.fc(out))


Test Accuracy of Real: 82% (219/266)
Test Accuracy of Fake: 11% (28/248)
Test Accuracy (Overall): 48% (247/514)
For test category horse:
Test Accuracy of Real: 90% (109/120)
Test Accuracy of Fake: 10% (14/140)
Test Accuracy (Overall): 47% (123/260)
For test category orange:
Test Accuracy of Real: 76% (189/248)
Test Accuracy of Fake:  1% ( 4/266)
Test Accuracy (Overall): 37% (193/514)
For test category summer:
Test Accuracy of Real: 98% (302/308)
Test Accuracy of Fake:  3% ( 8/238)
Test Accuracy (Overall): 56% (310/546)
For test category winter:
Test Accuracy of Real: 99% (236/238)
Test Accuracy of Fake:  7% (22/308)
Test Accuracy (Overall): 47% (258/546)
For test category zebra:
Test Accuracy of Real: 87% (123/140)
Test Accuracy of Fake: 20% (24/120)
Test Accuracy (Overall): 56% (147/260)
